In [ ]:
import os
from datasets import load_dataset 

# Define how much of the dataset you want to download
sample_size = 1_0 

# Use streaming to avoid loading the entire dataset into memory
dataset = load_dataset("openwebtext", streaming=True)

# Take only a subset of the training data
sampled_dataset = dataset["train"].take(sample_size)

# # Define where to save the downloaded subset
# output_dir = "openwebtext_sample"
# os.makedirs(output_dir, exist_ok=True)

# # Save the subset to disk
# sampled_dataset.save_to_disk(output_dir)

# print(f"Saved {sample_size} samples to {output_dir}")